# Helical Extension Spring

<table><tr>
<td>
<img src="./img/Helical_Extension_Spring/Extension_Spring_-_Figure_10-6.png" width=300"/> </td>
<td>
<img src="./img/Helical_Extension_Spring//Extension_Spring_-_Figure_10-7.png" width=550"/> </td>
</tr></table>

Importing the relevant Classes

In [1]:
from me_toolbox.springs import ExtensionSpring, Spring
from me_toolbox.tools import inch_to_millimetre as im
from me_toolbox.tools import millimetre_to_inch as mi
from me_toolbox.tools import lbs_to_newtons as pn
from me_toolbox.tools import lbs_per_in_to_newtons_per_mm as pinmm

## Geometric properties

d - Wire diameter<br>
OD - Spring outside diameter<br>
D - Spring mean diameter<br>
r1 - Hook's main radius<br>
r2 - Hook's secondary radius

In [2]:
d = im(0.035)
OD = im(0.248)
D = OD - d
r1 = im(0.106)
r2 = im(0.089)
print(f"{d = :.2f}mm\n"
      f"{OD = :.2f}mm\n"
      f"{D = :.2f}mm\n"
      f"{r1 = :.2f}mm\n"
      f"{r2 = :.2f}mm\n")

d = 0.89mm
OD = 6.30mm
D = 5.41mm
r1 = 2.69mm
r2 = 2.26mm



## Material properties

Table 10-4 from Shigley:
<table><tr>
<td>
<img src="./img/Helical_Compression_Spring/Tensile_strength_estimation.png" width=360"/> </td>
<td>
<img src="./img/Helical_Extension_Spring/Mechanical_properties_of_some_spring_wires_-_table_10-5.png" width=300"/> </td>
</tr></table>

Material options for use in Spring.material_prop method are:<br>
  * music wire<br>
  * oq&t wire<br>
  * hard-drawn wire<br>
  * chrome-vanadium wire<br>
  * chrome-silicon wire<br>
  * 302 stainless wire<br>
  * phosphore-bronze wire<br>

G - Shear modulus<br>
E - Elastic modulus<br>
Sut - Ultimate tensile strength

In [3]:
# music wire
G = 80e3  # [MPa]
E = 197.9e3  # [MPa]
Sut = Spring.material_prop('hard-drawn wire', d, metric=True, verbose=False)
print(f"Sut = {Sut:.2f}")

Sut = 1823.31


Table 10-6 from Shigley:
<table><tr>
<td> <img src="./img/Helical_Extension_Spring/Present_of_tensile_strength_-_Table_10-7.png" width=600"/> </td>
</tr></table>


In [4]:
body_torsion_yield_percent = 0.45
end_bending_yield_percent = 0.75
end_torsion_yield_percent = 0.4

## Forces and spring rate

k - Spring rate<br>
Fmax - Maximum force<br>
Fi - Initial force (force at free state from manufacturing process)

In [5]:
k = pinmm(17.912907845)  # [N/mm]
Fmax = pn(5.25)  # [N]
Fi = pn(1.19)  # [N]
print(f"{k = :.2f}\n"
      f"{Fmax = :.2f}\n"
      f"{Fi = :.2f}\n")

k = 3.13
Fmax = 23.35
Fi = 5.29



## Optional parameters

Shot peening - a cold working process in which the surface of the spring is impacted with a small spherical mediacalled shot in order to increase the fatigue resistance.<br>
rho - Material density

In [6]:
peened=True
rho = 7800  # [kg/m^3]

## Defining the spring

Required parameters:
  * max_force - The maximum force in $[N]$
  * initial_tension - The initial tension in $[N]$
  * wire_diameter - The wire diameter in $[mm]$
  * spring_diameter - The spring diameter in $[mm]$
  * hook_r1 - The spring's hook big radius in $[mm]$
  * hook_r2 - The spring's hook small radius in $[mm]$
  * ultimate_tensile_strength - The ultimate tensile strength in $[MPa]$
  * body_shear_yield_precent - the precentage of the shear yield strength(Ssy) from the ultimate tensile strength (Sut)
  * hook_normal_yield_percent - the precentage of the normal yield strength(Sy) from the ultimate tensile strength (Sut)
  * hook_shear_yield_percent - the precentage of the shear yield strength(Ssy) from the ultimate tensile strength (Sut)
  * shear_modulus - The material shear modulus in $[MPa]$
  * elastic_modulus - The material elastic modulus in $[MPa]$
  * spring_rate - The spring rate in $[N/mm]$

Optional parameters: <br>
(the bold parameters are the defaults)
  * shot_peened=**False** - If the spring underwent shot peening
  * density=**None** - The Material density in $[kg/mm^3]$

In [7]:
spring = ExtensionSpring(max_force=Fmax, 
                         initial_tension=Fi,
                         wire_diameter=d, 
                         spring_diameter=D, 
                         hook_r1=r2, hook_r2=r2,
                         ultimate_tensile_strength=Sut, 
                         body_shear_yield_percent=body_torsion_yield_percent, 
                         hook_normal_yield_percent=end_bending_yield_percent, 
                         hook_shear_yield_percent=end_torsion_yield_percent,
                         shear_modulus=G, elastic_modulus=E,
                         spring_rate=k, 
                         shot_peened=peened, 
                         density=rho)

### Calculated geometric properties

In [8]:
print(f"Free Length (L0) = {spring.free_length:.2f}\n"
      f"Active Coils (Na) = {spring.active_coils:.2f}\n"
      f"Body Coils (Nb) = {spring.body_coils:.2f}")

Free Length (L0) = 20.76
Active Coils (Na) = 12.58
Body Coils (Nb) = 12.18


### Calculated material properties

In [9]:
print(f"Shear yield strength (Ssy) = {spring.shear_yield_strength:.2f}\n"
      f"Shear ultimate strength (Ssu) = {spring.shear_ultimate_strength:.2f}\n")

Shear yield strength (Ssy) = 820.49
Shear ultimate strength (Ssu) = 1221.62



## Static analysis

In [10]:
ns = spring.static_safety_factor(verbose=False)
print(f"body safety factors: {ns['n_body']:.2f}\n"
      f"hook normal safety factors: {ns['n_hook_normal']:.2f}\n"
      f"hook shear safety factors: {ns['n_hook_shear']:.2f}")

body safety factors: 1.44
hook normal safety factors: 1.23
hook shear safety factors: 1.35


## Fatigue analysis

In [11]:
F_max = pn(5)  # [N]
F_min = pn(1.5)
n = spring.fatigue_analysis(max_force=F_max, min_force=F_min, reliability=50, verbose=True)
print(f"Body's fatigue safety factor (nf) = {n['nf_body']:.2f}\n"
      f"body's static safety factor (ns) = {n['ns_body']:.2f}\n"
      f"Hook's normal fatigue safety factor (ns) = {n['nf_hook_normal']:.2f}\n"
      f"Hook's normal Static safety factor (ns) = {n['ns_hook_normal']:.2f}\n"
      f"Hook's shear fatigue safety factor (ns) = {n['nf_hook_shear']:.2f}\n"
      f"Hook's shear Static safety factor (ns) = {n['ns_hook_shear']:.2f}\n")

print(f"The minimal safety factor is: {min(n, key=n.get)} = {n[min(n, key=n.get)]:.2f}")


Alternating force = 7.78, Mean force = 14.46

Body's alternating body shear stress = 180.65, Body's mean body shear stress = 335.50

Hook's alternating shear stress = 180.65, Hook's mean shear stress = 335.50

Hook's alternating normal stress = 370.17, Hook's mean normal stress = 687.46

Sut = 1823.31, Sse = 492.01, Se = 852.71, Ssu = 1221.62,
Ssy_body = 820.49, Ssy_hook = 729.32, Sy_hook = 1367.48

Body's fatigue safety factor (nf) = 1.95
body's static safety factor (ns) = 1.59
Hook's normal fatigue safety factor (ns) = 1.53
Hook's normal Static safety factor (ns) = 1.29
Hook's shear fatigue safety factor (ns) = 1.95
Hook's shear Static safety factor (ns) = 1.41

The minimal safety factor is: ns_hook_normal = 1.29


## Frequency analysis

The natural frequency must be greater than 20 times the working frequency<br>
$(f_{natural} > 20 \cdot f_{working})$

In [12]:
spring.natural_frequency(density=rho, working_frequency=0.5, verbose=True)

Note: the natural frequency=0.87 for fixed ends is not larger than 20*working frequency=10.00 
which means the spring can resonance

Note: the natural frequency=0.43 for one fixed and one free ends is not larger than 20*working frequency=10.00 
which means the spring can resonance



{'fixed-fixed': 0.869991487847817, 'fixed-free': 0.434995743923909}